# **PART II: Playing with Latent Codes**

---



### The repo we'll be using requires TF version 1.12 and Cuda 9.0...
### Thus we need to check and then downgrade tensorflow and cuda

In [0]:
import tensorflow as tf
tf.VERSION

'1.15.0'

In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


# Annoying part 1 of 2: Downgrade to TensorFlow 1.12.2

In [0]:
!pip install --upgrade tensorflow-gpu==1.12.2

     |████████████████████████████████| 127.8MB 89kB/s 
     |████████████████████████████████| 3.1MB 35.0MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.12.2 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


# Annoying part 2 of 2: Install Cuda 9.0 (this can take a few minutes, be patient)

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1
!echo ****** Cuda reinstall completed. Restart runtime now! *******

--2019-11-02 16:01:17--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.182.215
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.182.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?GckEcc3jQackBsmJGvwE99Q8sdds-uEPkRI84b-j2AxDcQRyQUNlK076f3awywo5tjIJNbXjc8BrS5ZNNzKUWt2YH3Z1VXKhldW1W3XW91Ksavxa503v2OvyR9PxQFsb5zS_mnUFH33r6Zhn-JPsIGOJZfqbij2FVeMqOHKqhvyFPNVTVWrXlS0X7eALQI-sX0y8CShJSh0bPqKki5yO [following]
--2019-11-02 16:01:18--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?GckEcc3jQackBsmJGvwE99Q8sdds-uEPkRI84b-j2AxDcQRyQUNlK076f3awywo5tjIJNbXjc8BrS5ZNNzKUWt2YH3Z1VXKhldW1W3XW91Ksav

## When the above cell has finished executing, restart the kernel runtime to reload everything

### Safety check to see if everything worked:

In [0]:
import tensorflow as tf
print("Now running TensorFlow version %s on Colab!" %tf.VERSION)
assert tf.VERSION == '1.12.2'

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

Now running TensorFlow version 1.12.2 on Colab!


In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:03_CDT_2017
Cuda compilation tools, release 9.0, V9.0.176


## If the above cells showed TensorFlow version 1.12.2 and Cuda release 9.0, you're good to go!

## Cloning InterFaceGAN

In [0]:
!git clone https://github.com/tr1pzz/InterFaceGAN.git

Cloning into 'InterFaceGAN'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 127 (delta 14), reused 124 (delta 14), pack-reused 0
Receiving objects: 100% (127/127), 2.06 MiB | 3.96 MiB/s, done.
Resolving deltas: 100% (14/14), done.


## Changing  directory to InterFaceGAN

In [0]:
cd /content/InterFaceGAN/

/content/InterFaceGAN


## Download the pretrained StyleGAN FFHQ network from NVIDIA:

In [0]:
!gdown https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ
!mv /content/InterFaceGAN/karras2019stylegan-ffhq-1024x1024.pkl /content/InterFaceGAN/models/pretrain/karras2019stylegan-ffhq-1024x1024.pkl

Downloading...
From: https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ
To: /content/InterFaceGAN/karras2019stylegan-ffhq-1024x1024.pkl
325MB [00:01, 168MB/s]


# Let us load our latent space vectors:
## Manually upload the output_vectors.npy file to the root of the directory

In [0]:
import numpy as np
final_w_vectors = np.load('/content/output_vectors.npy')

print("%d latent vectors of shape %s loaded from %s!" %(final_w_vectors.shape[0], str(final_w_vectors.shape[1:]), 'output_vectors.npy'))

2 latent vectors of shape (18, 512) loaded from output_vectors.npy!


# The InterFaceGAN comes with a bunch of pretrained latent directions
## Let's configure our latent-space interpolation
### (Change the settings below to morph the face)

In [0]:
latent_direction = 'age'        #### Pick one of ['age', 'eyeglasses', 'gender', 'pose', 'smile']
morph_strength = 3              # Controls how strongly we push the face into a certain latent direction (try 1-5)
nr_interpolation_steps = 48     # The amount of intermediate steps/frames to render along the interpolation path

In [0]:
print("python edit.py \
      -m stylegan_ffhq \
      -b boundaries/stylegan_ffhq_%s_w_boundary.npy \
      -s Wp \
      -i '/content/output_vectors.npy' \
      -o results/%s \
      --start_distance %.2f \
      --end_distance %.2f \
      --steps=%d" %(latent_direction, latent_direction, -morph_strength, morph_strength, nr_interpolation_steps))

python edit.py       -m stylegan_ffhq       -b boundaries/stylegan_ffhq_age_w_boundary.npy       -s Wp       -i '/content/output_vectors.npy'       -o results/age       --start_distance -3.00       --end_distance 3.00       --steps=48


# Run the latent space manipulation & generate images:

In [0]:
boundary_file = 'stylegan_ffhq_%s_w_boundary.npy' %latent_direction

print("Ready to start manipulating faces in the ** %s ** direction!" %latent_direction)
print("Interpolation from %d to %d with %d intermediate frames." %(-morph_strength, morph_strength, nr_interpolation_steps))
print("\nLoading latent directions from %s" %boundary_file)

Ready to start manipulating faces in the ** age ** direction!
Interpolation from -3 to 3 with 48 intermediate frames.

Loading latent directions from stylegan_ffhq_age_w_boundary.npy


# The code cell below has a bug I still need to fix...
## First time you run it, it will give an error.
## ----> Don't worry: Just run the same cell again and it should work :p


In [0]:
import subprocess
return_code = subprocess.call("rm -r results/%s" %latent_direction, shell=True)

run_command = "python edit.py \
      -m stylegan_ffhq \
      -b boundaries/stylegan_ffhq_%s_w_boundary.npy \
      -s Wp \
      -i '/content/output_vectors.npy' \
      -o results/%s \
      --start_distance %.2f \
      --end_distance %.2f \
      --steps=%d" %(latent_direction, latent_direction, -morph_strength, morph_strength, nr_interpolation_steps)


print("Running latent interpolations... This should not take longer than ~1 minute")
print("Running: %s" %run_command)
return_code = subprocess.call(run_command, shell=True)

if not return_code:
  print("Latent interpolation successfully dumped to disk!")
else:
  print("Something went wrong, try re-executing this cell...")

Running latent interpolations... This should not take longer than ~1 minute
Running: python edit.py       -m stylegan_ffhq       -b boundaries/stylegan_ffhq_age_w_boundary.npy       -s Wp       -i '/content/output_vectors.npy'       -o results/age       --start_distance -3.00       --end_distance 3.00       --steps=48
Latent interpolation successfully dumped to disk!


# Finally, turn the results into pretty movies!
### Adjust which video to render & at what framerate:

In [0]:
image_folder = '/content/InterFaceGAN/results/%s' %latent_direction
video_fps = 8.

## Redner the videos:

In [0]:
from moviepy.editor import *
import cv2

out_path = '/content/output_videos/'

images = [img_path for img_path in sorted(os.listdir(image_folder)) if '.jpg' in img_path]
os.makedirs(out_path, exist_ok=True)

prev_id = None
img_sets = []
for img_path in images:
  img_id = img_path.split('_')[0]
  if img_id == prev_id: #append
    img_sets[-1].append(img_path)
    
  else: #start a new img set
    img_sets.append([])
    img_sets[-1].append(img_path)
  prev_id = img_id

print("Found %d image sets!\n" %len(img_sets))
if image_folder[-1] != '/':
  image_folder += '/'

def make_video(images, vid_name):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(vid_name, fourcc, video_fps, (1024, 1024))
    gen = {}
    for img in images:
      video.write(img)
    video.release()
    print('finished '+ vid_name)
    
    
for i in range(len(img_sets)):
  print("############################")
  print("\nGenerating video %d..." %i)
  set_images = []
  vid_name = out_path + 'out_video_%s_%02d.mp4' %(latent_direction,i)
  for img_path in img_sets[i]:
    set_images.append(cv2.imread(image_folder + img_path))

  set_images.extend(reversed(set_images))
  make_video(set_images, vid_name)

Found 2 image sets!

############################

Generating video 0...
finished /content/output_videos/out_video_age_00.mp4
############################

Generating video 1...
finished /content/output_videos/out_video_age_01.mp4


# What did we get?
Navigate to output_videos/

(you might have to "REFRESH" the Filebrowser)

Download the videos to your local pc